<a href="https://colab.research.google.com/github/Rudra-Codes/Couert_Agent/blob/main/Main_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read)

In [2]:
import os
import pandas as pd
from typing import List, Dict
from huggingface_hub import InferenceClient

In [14]:
class agents:
  def __init__(self,
               name: str,
               system_prompt: str,
                 model: str = "microsoft/Phi-3-mini-4k-instruct"):
    self.name = name
    self.system_prompt = system_prompt
    self.client = InferenceClient(
        model,
        token=os.getenv("HF_API_TOKEN"))

  def response(self,dialogue = None,role_msg: str = None,result: bool = False) :
    prompt = f"<|system|>\n{self.system_prompt}\n"
    if result:
      prompt += f"<|User|>\n{role_msg}\n"
      prompt += "<|assistant|>\n"
    else :
      prompt += "<|user|\nDialouges so far:\n"
      for m in dialogue:
        prompt += f"{m['role']}: {m['content']}\n"
      if role_msg != None:
        prompt += role_msg
      prompt += "<|assistant|>\n"

    answer = self.client.text_generation(
        prompt,
        max_new_tokens=512,
        temperature=0.9,
        do_sample = True,
        stream=False,
    )
    answer = answer.strip()
    # Saving answer

    dialogue.append({"role": self.name, "content": answer})
    return answer

In [4]:
class Resulter:
  def __init__(self,
               name: str,
                 model: str = "microsoft/Phi-3-mini-4k-instruct"):
    self.name = name
    self.client = InferenceClient(
        model,
        token=os.getenv("HF_API_TOKEN"))
  def response(self,role_msg):
    prompt = f"<|system|>\n{role_msg}"
    prompt += "\nIs the plea accepted? Answer with 1 or 0.\n"
    prompt += "<|assistant|>\n"
    answer = self.client.text_generation(
        prompt,
        max_new_tokens=512,
        temperature=0.2,
        do_sample=False,
        stream=False,
    )
    answer = answer.strip()
    return answer


In [ ]:
with open('data_reduced.csv', 'r', encoding='utf-8') as file:
    lines = file.readlines()

case_number = 1 # Specify the which case from data

case_background = lines[case_number].strip()

In [5]:
judge_prompt = """Court Simulation with case details are:
You are the judge of the court and your role is to decide who will speak next in discussion.
Other roles are:
Defendant = 1
Defense Lawyer = 2
Plaintiff = 3
Prosecution Lawyer = 4
Call Judger = 5 if case is completed
"""

defender_prompt = """
The defender provides legal advice to the accused person throughout the legal process.
The defender investigates the facts of the case and gathers evidence to support the defense.
They build and present a defense strategy aimed at proving the defendant’s innocence or reducing their liability.
During the trial, the defender cross-examines the prosecution’s witnesses and objects to any improper procedures or evidence.
They ensure that the defendant’s constitutional rights, such as the right to remain silent and the right to a fair trial, are protected at all times.
The defender may also negotiate plea bargains with the prosecution if it serves the best interest of the defendant.
Throughout the case, the defender must act in the defendant’s best interest while upholding ethical and legal standards.
"""

laintiff_prompt = """
The plaintiff is the party who initiates a legal case by filing a complaint against another party, known as the defendant.
The plaintiff claims to have been wronged or harmed and seeks a legal remedy from the court.
They are responsible for presenting evidence and arguments to prove that the defendant is legally responsible for the harm caused.
The plaintiff works with their attorney to develop a case strategy, identify key witnesses, and gather supporting documents or materials.
During the trial, the plaintiff presents their case first, introduces evidence, and questions witnesses to support their claims.
The plaintiff must meet the burden of proof, which typically means proving their case by a preponderance of the evidence in civil cases.
Throughout the proceedings, the plaintiff aims to convince the judge or jury to grant relief, which could include compensation, an injunction, or other court-ordered actions.
"""

defense_lawyer_prompt = """
You are **Alex Carter**, lead *defense counsel*.
Goals:
• Protect the constitutional rights of the defendant.
• Raise reasonable doubt by pointing out missing evidence or alternative explanations.
• Be respectful to the Court and to opposing counsel.
Style:
• Crisp, persuasive, grounded in precedent and facts provided.
• When citing precedent: give short case name + year (e.g., *Miranda v. Arizona* (1966)).
Ethics:
• Do not fabricate evidence; admit uncertainty when required.
"""

prosecution_lawyer_prompt = """
You are **Jordan Blake**, *Assistant District Attorney* for the State.
Goals:
• Present the strongest good‑faith case against the accused.
• Lay out facts logically, citing exhibits or witness statements when available.
• Anticipate and rebut common defense arguments.
Style:
• Formal but plain English; persuasive, with confident tone.
Ethics:
• Duty is to justice, not merely to win. Concede points when ethically required.
"""

judger_prompt = """
You are a judge of the court.
Predict the verdict of the case based on case details provided wether the case can be granted or not.
"""

result_prompt = """
Output 1 if granted and 0 if not.
"""

In [15]:
dialogue = []
final_answer = []
dialogue.append({"role": "Judge", "content": "Provide the opening statement"})
print(dialogue[0]["content"],"\n")

Provide the opening statement 



In [ ]:
judge = agents("Judge", judge_prompt)
prosecution_lawyer = agents("Prosecution Lawyer", prosecution_lawyer_prompt)
defense_lawyer = agents("Defense Lawyer", defense_lawyer_prompt)
plaintiff = agents("Plaintiff", laintiff_prompt)
defender = agents("Defendant", defender_prompt)

verdict = judge.response(dialogue,"Just give the integer of role who will speak based on the discussion")

# print(verdict[0])
# print(verdict)

In [ ]:
verdict_bool : bool = True
while (verdict_bool) :
  verdict_bool = False
  if (verdict.split()[0] == "Defendant") or (verdict[0] == 1):
    print(defender.response(dialogue,verdict),"\n")
    verdict = judge.response(dialogue,"Just give the integer of role who will speak based on the discussion")
    verdict_bool = True
  elif (verdict.split()[0] == "Plaintiff") or (verdict[0] == 3):
    print(plaintiff.response(dialogue,verdict),"\n")
    verdict = judge.response(dialogue,"Just give the integer of role who will speak based on the discussion")
    verdict_bool = True
  elif (verdict.split()[0] == "Defense") or (verdict[0] == 2):
    print(defense_lawyer.response(dialogue,verdict),"\n")
    verdict = judge.response(dialogue,"Just give the integer of role who will speak based on the discussion")
    verdict_bool = True
  elif (verdict.split()[0] == "Prosecution") or (verdict[0] == 4):
    print(prosecution_lawyer.response(dialogue,verdict),"\n")
    verdict = judge.response(dialogue,"Just give the integer of role who will speak based on the discussion")
    verdict_bool = True

In [16]:
data = pd.read_csv("cases.csv")
for i in range(0,49) :
  case_data = data.iloc[i][2]
  case_data = case_data[:3000].replace("\n", "")
  # print (case_data)
  judger = agents("Judge", judger_prompt)
  verdict = judger.response(dialogue, f"Case Details: {case_data}", True)
  print(verdict)

# Using Another llm to judge verdict
  resulter = Resulter("resulter")
  answer = resulter.response(verdict)
  # print(answer)
  final_answer.append(answer[0])

  # ----------Or Usinf pipeline sentimental analysis to judge verdict-----------
  # classifier = pipeline("sentiment-analysis")
  # answer = classifier(verdict[:512])  # Or any text input
  # answer = 0 if answer[0]["label"] == 'positive' else 1
  # final_answer.append(answer)

<ipython-input-16-c8589e0fcfd4>:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  case_data = data.iloc[i][2]


It appears that the respondent in the appeal, Parmanand, is being accused of falsely marking the attendance of another employee, Ashok Kumar, in the company's payroll and preparing a bogus identity card for this purpose. The inquiry officer conducted the proceedings under the Punjab Government Servants Company Rules 1966, and the accused is charged with a failure to maintain integrity and devotion to duty.

In a civil appeal case like this, the central administrative tribunal (CATCH) has the power to examine the adequacy of the penalty awarded to a government servant in disciplinary proceedings. The appellant, instead of challenging the disciplinary action itself, seeks a review of the penalty, arguing that it was not adequate.

To predict the verdict, we need to analyze the fundamental aspects of the case, and consider the following key factors:

1. Evidence: Was there sufficient evidence to support the charges against the respondent? The tribunal must assess the credibility and weigh

HfHubHTTPError: 402 Client Error: Payment Required for url: https://router.huggingface.co/hf-inference/models/microsoft/Phi-3-mini-4k-instruct (Request ID: Root=1-68049124-4ca785b07efda10726d60c71;a21bded3-2485-4b1a-bc4c-bd53d6f70c9f)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.

In [17]:
print(final_answer)

['1', '1', '0', '1', '1', '0', '0', '0', '1', '0', '0', '0', 'I', '0', '0', '0']


In [ ]:
df = pd.DataFrame({
    "id": data.iloc[:50,1],
    "verdict": final_answer
})
df.to_csv('final_answer.csv', index=False)

In [ ]:
import gc
import torch

gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()